In [ ]:
import os
import scipy.io as scipyio
import numpy as np
from collections import defaultdict
import experiment
import tensorflow as tf

In [ ]:
strategies = ["Sparsemax","Sparsemax","Sparsemax"]
backuprules = ["SoftBellman","SparseBellman","Bellman"]
scales = [1, 0.1, 0.01]
scales_name = ["high","mid","low"]

action_res_list = [[51,51], [31,31], [11,11], [3,3]]
action_res_name = ["large","midlarge","midsmall","small"]

env_name = "Reacher-v1"
dqn_hidden_spec = [
                {'dim': 512,'activation': tf.nn.tanh},
                {'dim': 512,'activation': tf.nn.tanh}
            ]

n_exp = 5
n_epi = 1500
n_eval = 100

data = defaultdict(lambda : defaultdict(lambda : defaultdict(lambda : defaultdict(lambda : None))))
for action_res, action_name in zip(action_res_list,action_res_name):
    for scale, scale_name in zip(scales, scales_name):
        for strategy, backuprule in zip(strategies, backuprules):
            print("Problem: {}, Action: {}, Temp: {}, Strategy: {}, Backup: {}".format(env_name,np.prod(action_res),scale,strategy,backuprule))
            return_list = np.zeros((n_exp,n_epi))
            eval_list = np.zeros((n_exp,n_eval))
            for i_exp in range(n_exp):
                exp = experiment.Experiments(seed=i_exp, env_name = 'Reacher-v1', action_res=action_res,
                                         dqn_hidden_spec=dqn_hidden_spec, batch_size = 512, learning_rate=1e-4,
                                         discount = 0.99, max_epi = n_epi, max_step = 10, target_update_period = 25,
                                         replay_memory_size = 50000, eps_decay_rate=0.999, 
                                         scale=scale, strategy=strategy, backuprule=backuprule)
                print("{}/{} Experiment Start".format(i_exp,n_exp))
                return_list[i_exp,:] = exp.run(display_period=n_epi+1)
                eval_list[i_exp,:] = exp.evaluation(max_eval_epi=n_eval)

            data[action_name][scale_name][strategy][backuprule]  = {"return_list":return_list,"eval_list":eval_list}

data_file_name = env_name+"_sparsemax.mat"
scipyio.savemat(data_file_name, data)
print("{} is finished and is saved".format(data_file_name))

[2017-11-22 21:05:29,951] Making new env: Reacher-v1


Problem: Reacher-v1, Action: 2601, Temp: 1, Strategy: Sparsemax, Backup: SoftBellman
0/5 Experiment Start
